In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timewarp_lib.vector_timewarpers as vtw
import torch
import torch.nn as nn

In [ ]:
tau = 1
yzero = 0

# arbitrary?
alpha_z = 10
beta_z = alpha_z/4

N = 50
sigma = 2/N

def phase_func(t):
    phasefactor = -np.log(0.01)
    return np.exp(-t * phasefactor)

cs = phase_func(np.linspace(0,1,N,endpoint=True))

fulldat = np.load("../data/trainTest2DLetterACache.npz")
fulldat["train"].shape
dat = fulldat["train"]
numdims = dat.shape[2]
numts = dat.shape[1]
numtrajs = dat.shape[0]


ts = np.linspace(0,1,numts)
xs = phase_func(ts)

In [ ]:
# Yeah, i don't _really_ get it, but an original Ijspeert paper
# (and the Learning Parametric Dynamic Movement... paper)
# have the forcing term affect velocity, not acceleration
def numeric_integration(ydemos, ts, tau, g, alpha_z, beta_z):
    step_size = 0.00001
    i = 0
    t = ts[i]
    i += 1
    z = 0
    zs = []
    zs.append(z)
    while i < len(ts):
        while i < len(ts) and t < ts[i]:
            interp_frac = (t - ts[i-1])/(ts[i] - ts[i-1])
            y = (1-interp_frac) * ydemos[i-1] + interp_frac * ydemos[i]
            z += alpha_z * (beta_z * (g - y) - z)/tau * step_size
            t += step_size
        zs.append(z)
        i += 1
    return np.array(zs)

In [ ]:
# basisphis, targetfunction, xs are all evaluated at ts
def fit_target_i(i, basisphis, targetfunction, xs, yzero, g):
    s = xs * (g - yzero)
    gamma = np.diag(basisphis[i])
    # equation 2.14 from Dynamical Movement Primitives: Learning Attractor Models for Motor Behaviors
    numerator = s @ gamma @ targetfunction
    denominator = s @ gamma @ s
    return numerator / denominator

In [ ]:
def simulate(fitted_f, ts, g, alpha_z, beta_z):
    step_size = 0.00001
    i = 0
    t = ts[i]
    ys = [] # position
    zs = [] # velocity
    y = 0
    z = 0
    t = 0
    while i < len(ts):
        while i < len(ts) and t < ts[i]:
            interp_frac = (t - ts[i-1])/(ts[i] - ts[i-1])
            f = (1-interp_frac) * fitted_f[i-1] + interp_frac * fitted_f[i]
            z += alpha_z * (beta_z * (g - y) - z)/tau * step_size
            y += (z + f)/tau * step_size
            t += step_size
        ys.append(y)
        zs.append(z)
        i += 1
    return (np.array(ys), np.array(zs))

## Compute DTW parameterization of each training trajectory

In [ ]:
# traj should be 2d
def compute_dtw_parameterization(traj):
    assert len(traj.shape) == 2, "traj should be 2d...just send in one"
    start_offset_0, start_offset_1 = traj[0]
    ydemos = traj-traj[0:1]
    result_dictionary = {}
    for dim in range(2):
        ydemo = ydemos[:,dim]
        ydemoprime = (ydemo[2:]-ydemo[:-2])/(ts[1]-ts[0])/2
        ydemoprime = np.concatenate(((ydemo[1:2]-ydemo[:1])/(ts[1]-ts[0]),ydemoprime,(ydemo[-1:]-ydemo[-2:-1])/(ts[1]-ts[0])))
        yzero = ydemo[0]
        g = ydemo[-1]
        basisphis = np.array([np.exp(-(phase_func(ts) - c)**2/((sigma * c)**2)) for c in cs])
        zdemo = numeric_integration(ydemo, ts, tau, g, alpha_z, beta_z)
        ftarget = tau * ydemoprime - zdemo
        ws = np.array([fit_target_i(i, basisphis, ftarget, xs, yzero, g) for i in range(len(basisphis))])
        result_dictionary[f"ws_{dim}"] = ws
        result_dictionary[f"g_{dim}"] = g
    result_dictionary[f"start_offset_0"] = start_offset_0
    result_dictionary[f"start_offset_1"] = start_offset_1
    return result_dictionary

In [ ]:
important_results_object = []
for i in range(len(dat)):
    if i % 10 == 0:
        print(i)
    result_dictionary = compute_dtw_parameterization(dat[i])
    important_results_object.append(result_dictionary)
    
parameters_vector = np.array([np.concatenate((dic["ws_0"],dic["ws_1"],
                                    [dic["start_offset_0"],dic["start_offset_1"],
                                    dic["g_0"],dic["g_1"]]
                                   ))
                     for dic in important_results_object])

np.save("parameters_vector_dmps.npy", parameters_vector)

In [ ]:
def decode_parameter_vector(parameter_vector):
    ws_0 = parameter_vector[:N]
    ws_1 = parameter_vector[N:2*N]
    start_offset_0,start_offset_1,g_0,g_1 = parameter_vector[2*N:]
    basisphis = np.array([np.exp(-(phase_func(ts) - c)**2/((sigma * c)**2)) for c in cs])
    
    positions = []
    for g,ws,start_offset in [(g_0,ws_0,start_offset_0), (g_1,ws_1,start_offset_1)]:
        yzero=0 # centered training data all starts at zero
        fitted_f = np.einsum("it,i->t",basisphis,ws)/np.einsum("it->t",basisphis) * xs * (g-yzero)
        # ys = position, zs = velocity
        ys,zs = simulate(fitted_f,ts,g,alpha_z,beta_z)
        positions.append(ys + start_offset)
    positions=np.array(positions).T
    return positions

In [ ]:
def train_dmp_parameter_model(training_data, latent_dim,scale_last_four_dims=1):
    num_trajs, num_channels = training_data.shape
    print(f"we have num_trajs:{num_trajs}, num_channels:{num_channels}")
    flattened_trajs = np.copy(training_data)
    
    flattened_trajs[:,-4:] = flattened_trajs[:,-4:] * scale_last_four_dims

    # compute and remove the mean trajectory (mean trajectory is of shape (num_timesteps * num_channels))
    mean_traj = np.mean(flattened_trajs, axis=0)
    centered_trajs = flattened_trajs - mean_traj[np.newaxis,:]

    # compute the PCA model using SVD
    u, s, vt = np.linalg.svd(centered_trajs)

    # keep only the first latent_dim number of dimensions
    if len(s) < latent_dim:
        raise Exception(f"You can't build a model of dimension {latent_dim} on a dataset with dimensionality {len(s)}")

    # the singular values written in matrix form
    smat = np.diag(s[:latent_dim])
    smatinv = np.diag(1./s[:latent_dim])
    # the first latent_dim directions of variation
    basis_vectors = vt[:latent_dim,:]
    # given a trajectory, use this to find its value in the latent space 
    # note that the dimensions of the embedding matrix are (latent_dim, nt*nc)
    embedding_matrix = smatinv @ basis_vectors 
    # note that we include the s factor here so that our latent space is roughly the unit gaussian ball :brain:
    # note further that the dimensions of the reconstruction matrix are also (latent_dim, nt*nc)
    reconstruction_matrix = smat @ basis_vectors

    return {
      "num_trajs" : num_trajs, 
      "num_channels" : num_channels,
      "latent_dim" : latent_dim,
      "mean_traj" : mean_traj,
      "embedding_matrix" : embedding_matrix,
      "reconstruction_matrix" : reconstruction_matrix}

In [ ]:
# Create a ModelApplier object based purely on a
# directory containing model information
class ModelApplier(object):
    def __init__(self, modeldata):
        self.num_trajs = modeldata["num_trajs"]
        self.num_channels = modeldata["num_channels"]
        self.latent_dim = modeldata["latent_dim"]
        self.mean_traj = modeldata["mean_traj"]
        self.embedding_matrix = modeldata["embedding_matrix"]
        self.reconstruction_matrix = modeldata["reconstruction_matrix"]

    # The input data should be of the shape
    # (num_apply_trajs, apply_latent_dim)
    def apply(self, data):
        num_apply_trajs, apply_latent_dim = data.shape
        if apply_latent_dim != self.latent_dim:
              raise Exception(f"The number of latent dim coordinates given: {apply_latent_dim} was not the same as the {self.latent_dim} expected by the model")

        # compute the linear combination of basis vectors 
        traj_offset_vectors = data @ self.reconstruction_matrix
        mean_vector = self.mean_traj.reshape(1,self.num_channels)
        # add back the mean vector
        traj_vectors = traj_offset_vectors + mean_vector
        # reshape to the official standard expected shape
        # (num_apply_trajs, num_timesteps, num_channels)
        result_trajs = traj_vectors.reshape(num_apply_trajs, self.num_channels)
        return result_trajs

    # The input data should be of the shape
    # (num_apply_trajs, self.num_timesteps, self.num_channels)
    # The output latent dimensions are of the shape
    # (num_apply_trajs, self.latent_dim)
    def embed(self, data):
        num_apply_trajs, apply_num_channels = data.shape
        assert apply_num_channels == self.num_channels, "channels must match for PCA"

        flattened_trajs = data
        mean_vector = self.mean_traj.reshape(1,self.num_channels)
        centered_trajs = flattened_trajs - mean_vector
        # the dimensions of the embedding matrix are (latent_dim, nt*nc)
        latent_vals_mat = centered_trajs @ self.embedding_matrix.T
        return(latent_vals_mat)


In [ ]:
test_time_dtw_vector_timewarper = vtw.DTWVectorTimewarper()

In [ ]:
def round_trip_loss(ma, traj, i,scale_last_four_dims=1):
    assert len(traj.shape) == 2, "pass in one traj at a time"
    num_ts, channels = traj.shape
    assert channels == 2, "2 channels for handwriting"
    dic = compute_dtw_parameterization(traj)
    params = np.concatenate((dic["ws_0"],dic["ws_1"],
                                    [dic["start_offset_0"],dic["start_offset_1"],
                                    dic["g_0"],dic["g_1"]]
                                   )).reshape(1,-1)
    params[:,-4:] = scale_last_four_dims * params[:,-4:]
    new_parameters = ma.apply(ma.embed(params))
    new_parameters[0][-4:] = new_parameters[0][-4:]/scale_last_four_dims
    pos = decode_parameter_vector(new_parameters[0])
    recon_train = pos.reshape((1,num_ts, channels))
    train = traj.reshape((1,num_ts, channels))
    
    train_dtw_recon, train_dtw_actual = test_time_dtw_vector_timewarper.timewarp_first_and_second(
        torch.tensor(recon_train,dtype=torch.float), 
        torch.tensor(train,dtype=torch.float))
    train_aligned_loss = (
        nn.functional.mse_loss(train_dtw_recon, train_dtw_actual, reduction="sum").detach().numpy()
        / (num_ts))
    train_error = np.sum(np.square(recon_train - train))/(num_ts)
    
    squareresults = (ma.latent_dim, train_aligned_loss, train_error, i)
    return squareresults

In [ ]:
for latent_dim in range(1,5):
    vals = train_dmp_parameter_model(parameters_vector,latent_dim,scale_last_four_dims=100)
    np.savez(f"dmpmodels/parametric_dmp_{latent_dim}.npz", dic=vals)

In [ ]:
# we trained on augmented data, but we should just apply to regular data

DATAFILE=f"../data/trainTest2DLetterARescaled.npz"
data = np.load(DATAFILE)
test = data["test"]
train = data["train"]

num_trains, num_ts, channels = train.shape
num_tests, num_ts, channels = test.shape

for latent_dim in range(1,5):
    vals = np.load(f"dmpmodels/parametric_dmp_{latent_dim}.npz",allow_pickle=True)["dic"].item()
    ma = ModelApplier(vals)
    all_results = []
    for dataset in [train, test]:
        print("running a dataset")
        square_losses = []
        for i in range(len(dataset)):
            if i % 10 == 0:
                print(i)
            square_losses.append(round_trip_loss(ma, dataset[i], i,scale_last_four_dims=100))
        square_losses = np.array(square_losses)
        all_results.append(square_losses)
    np.savez(f"intermediate_dmp_error_results_{latent_dim}.npz",train=all_results[0], test=all_results[1])

In [ ]:
final_results = []
for latent_dim in range(1,17):
    intermediate_results = np.load(f"intermediate_dmp_error_results_{latent_dim}.npz")
    valid_inds = intermediate_results["train"][:,0] == latent_dim
    if np.sum(valid_inds) != 125:
        continue
    ld,train_aligned_loss, train_error, checkval = np.mean(intermediate_results["train"][valid_inds,],axis=0)
    assert ld == latent_dim, "check your latent_dim"
    assert checkval == 62, "should have indices ranging from 0 to 124"
    
    valid_inds = intermediate_results["test"][:,0] == latent_dim
    latent_dim,test_aligned_loss, test_error, checkval = np.mean(intermediate_results["test"][valid_inds,],axis=0)
    assert ld == latent_dim, "check your latent_dim"
    assert checkval == 62, "should have indices ranging from 0 to 124"
    
    final_results.append((latent_dim, np.sqrt(train_aligned_loss),np.sqrt(test_aligned_loss), 
                       np.sqrt(train_error), np.sqrt(test_error)))
final_results = np.array(final_results)
np.savez("dmp_results.npy")

In [ ]:
final_results

In [ ]:
pos1 = decode_parameter_vector(parameters_vector[10])
pos2 = decode_parameter_vector((parameters_vector[15] + parameters_vector[10])/2)
pos3 = decode_parameter_vector(parameters_vector[15])

In [ ]:
for pos in [pos1, pos2, pos3]:
    plt.plot(pos[:,0],pos[:,1])